In [15]:
import pandas as pd
import os
import datetime as dt

In [34]:
def read_and_clean(country, lng_path, area_suffix):
        os.chdir(os.path.join("/Users","ethan","Dev","Thesis Work"))
        # start by iterating and reading in all of the files
        price_data = []
        #os.chdir("../..")
        os.chdir(os.path.join("Data", country, "Price"))
        for file in os.listdir():
                price_data.append(pd.read_csv(file))

        prices = pd.concat(price_data)

        # general clean up
        prices = prices.dropna(subset=["Day-ahead Price [EUR/MWh]"])
        prices = prices[prices["Day-ahead Price [EUR/MWh]"] != "n/e"]
        prices = prices[prices["Day-ahead Price [EUR/MWh]"] != "-"]
        prices["Price"] = prices["Day-ahead Price [EUR/MWh]"].apply(float)
        prices = prices[["MTU (CET/CEST)","Price"]]

        # convert the time range into a start + end time
        prices["Start"] = prices["MTU (CET/CEST)"].apply(lambda x: x.split("-")[0])
        prices["End"] = prices["MTU (CET/CEST)"].apply(lambda x: x.split("-")[1])


        ## READ AND CLEAN LOAD DATA
        # work through the same process for the load data
        load_data = []
        os.chdir(os.path.join("..", "Load"))
        for load_file in os.listdir():
                df = pd.read_csv(load_file)
                load_data.append(df)
        os.chdir(os.path.join("..",".."))
        loads = pd.concat(load_data)

        # special step for Germany due to some quirks of reporting
        if(country == "Germany"):
                loads["Forecast"] = loads["Day-ahead Total Load Forecast [MW] - BZN|DE-AT-LU"].combine_first(loads["Day-ahead Total Load Forecast [MW] - BZN|DE-LU"])
                loads["Actual"] = loads["Actual Total Load [MW] - BZN|DE-AT-LU"].combine_first(loads["Actual Total Load [MW] - BZN|DE-LU"])
        else:
                loads["Forecast"] = loads["Day-ahead Total Load Forecast [MW] - {}".format(area_suffix)]
                loads["Actual"] = loads["Actual Total Load [MW] - {}".format(area_suffix)]
        loads = loads.dropna(subset=["Forecast", "Actual"])
        loads = loads[loads["Actual"] != "n/e"]
        loads = loads[loads["Actual"] != "-"]
        loads["Actual"] = loads["Actual"].apply(float)
        loads["Forecast"] = loads["Forecast"].apply(float)

        loads = loads[["Time (CET/CEST)", "Forecast", "Actual"]]
        # make the time range effective
        loads["Start"] = loads["Time (CET/CEST)"].apply(lambda x: x.split("-")[0])
        loads["End"] = loads["Time (CET/CEST)"].apply(lambda x: x.split("-")[1])

        ## MERGING THE TWO DATASETS
        # merge the two sets of data
        elec_merged = pd.merge(left = prices, right = loads, on = "Start", how="left")
        elec_merged = elec_merged[["Start", "Price", "Actual", "Forecast"]]
        elec_merged["Date"] = pd.to_datetime(pd.to_datetime(elec_merged["Start"]).dt.date)
        ## READ IN LNG DATA FROM BLOOMBERG
        lng_data = pd.read_excel(lng_path)
        # filter columns
        lng_data = lng_data[["Date", "Last Price"]]
        # add the additional day to match up with the day ahead data
        lng_data["Date"] = pd.to_datetime(lng_data["Date"] + dt.timedelta(days=1))
        combined = elec_merged.merge(lng_data, on="Date")
        
        combined.to_csv("./{}/combined_data.csv".format(country))

        return combined


In [35]:
read_and_clean("Spain", "/Users/ethan/Dev/Thesis Work/Data/Bloomberg/SVB Day-Ahead Historical.xlsx", "BZN|ES")

,Start,Price,Actual,Forecast,Date,Last Price
0,01.01.2017 00:00,58.82,24401.0,24255.0,2017-01-01,24.50
1,01.01.2017 01:00,58.23,23393.0,23326.0,2017-01-01,24.50
2,01.01.2017 02:00,51.95,21909.0,22287.0,2017-01-01,24.50
3,01.01.2017 03:00,47.27,20615.0,21038.0,2017-01-01,24.50
4,01.01.2017 04:00,45.49,19628.0,20079.0,2017-01-01,24.50
...,...,...,...,...,...,...
59383,30.12.2015 19:00,50.60,31364.0,31359.0,2015-12-30,19.25
59384,30.12.2015 20:00,52.69,31393.0,31472.0,2015-12-30,19.25
59385,30.12.2015 21:00,51.10,30951.0,31481.0,2015-12-30,19.25
59386,30.12.2015 22:00,47.69,29126.0,29732.0,2015-12-30,19.25
